In [194]:
#Step 1: Import modules and connect to the database
import pandas as pd
import numpy as np
import pyodbc
import warnings
warnings.filterwarnings('ignore')

from IPython.display import HTML
config = dict(server='ABR-RIPLEYA-D1',
    port=      1433,
    database=  'neumodxRawDataDB',
    )

conn_str = ('SERVER={server};' +
            'Database={database};' +
            'TRUSTED_CONNECTION=yes')

conn = pyodbc.connect(
    r'DRIVER={SQL Server};' +
    conn_str.format(**config)
    )

In [195]:
##Step 2:  Load Consolidated Data from Gen 1.5 DB 


environment='HPV_RPT-8430_Original_Data_3'

def getSQLData_summary_coc(environment):
    query = ("Select * from [dbo].[summary_coc]"+
             " where  [Environment] like '%"+environment+"%'")
    print("Query:", query)
    df = pd.read_sql(query, conn)
    print("Length of Raw Data Frame: "+ str(len(df)))
    
    df = df.loc[~df['N500 Serial Number'].isnull()]
    print(df['N500 Serial Number'].unique())
    df.drop_duplicates(subset=['Sample ID', 'Start Date/Time'],inplace=True)
    print("Length of Filtered Data Frame: "+ str(len(df)))
    return df




OriginalData = getSQLData_summary_coc(environment)


Query: Select * from [dbo].[summary_coc] where  [Environment] like '%HPV_RPT-8430_Original_Data_3%'
Length of Raw Data Frame: 19765
['23000021' '96000005' 'N000007' 'N000012' '96000007' 'N000009']
Length of Filtered Data Frame: 5998


In [196]:
##Step 3:  Change Connection to NIMS (Gen 2) Database
from IPython.display import HTML
config = dict(server='ABR-RIPLEYA-L1',
    port=      1433,
    database=  'NIMS_New',
    )

conn_str = ('SERVER={server};' +
            'Database={database};' +
            'TRUSTED_CONNECTION=yes')

conn = pyodbc.connect(
    r'DRIVER={SQL Server};' +
    conn_str.format(**config)
    )

In [197]:
##Step 4: Get Final Data (1.9.2.6 Data processed with ADF 4.2.1) 

def get_NIMS_ChainOfCustody(trialid):
    query = "SELECT SampleTrials.SampleUId, Samples.[Sample ID], ChainOfCustodySets.[Start Date/Time] FROM ChainOfCustodySets Left JOIN SampleTrials ON SampleTrials.SampleUId=ChainOfCustodySets.SampleUId AND SampleTrials.TrialId = '"+trialid+"' Left JOIN Samples ON ChainOfCustodySets.SampleUId = Samples.UId ;"
    df = pd.read_sql(query, conn)
    return df

NIMSData = get_NIMS_ChainOfCustody('BC4699B0-DCB0-4ED2-ABCB-17628D7DEC22')

In [198]:
##Step 5: Set Index to Shared Properties between Original & NIMS Data
NIMSData.set_index(['Sample ID', "Start Date/Time"], inplace=True)
OriginalData.set_index(['Sample ID', "Start Date/Time"], inplace=True)

In [199]:
##Step 6: Combined Original Data and NIMS Data so that ChainOfCustodyInfo has SampleUID Column
ChainOfCustodyInfo = NIMSData.join(OriginalData)

In [200]:
consumables_list = ['Capture Plate Barcode',
 'Buffer Barcode',
 'Test Strip NeuMoDx Barcode',
 'Pcr Cartridge Barcode',
 'Release Reagent Barcode',
 'Wash Reagent Barcode']

for col in consumables_list:
    ChainOfCustodyInfo[col.replace('Barcode', 'Lot')] = ChainOfCustodyInfo[col].str[18:24]
    ChainOfCustodyInfo[col.replace('Barcode', 'Serial')] = ChainOfCustodyInfo[col].str[27:32]

In [202]:
[x for x in ChainOfCustodyInfo if "Software" in x]

['Software Version', 'Hamilton Software Version']

In [203]:
ChainOfCustodyInfo = ChainOfCustodyInfo[['SampleUId', 'Test Guid', 'Software Version', 'Assay Version', 'N500 Serial Number', 'PCR Module Serial', 'Pcr Cartridge Lane', 'Pcr Cartridge Barcode']+
                   [x.replace('Barcode', 'Lot') for x in consumables_list]+
                   [x.replace('Barcode', 'Serial') for x in consumables_list]]

In [204]:
ChainOfCustodyInfo

SampleUId  \
Sample ID Start Date/Time                                                     
-20Neg41  2022-04-26 08:26:01.2280000  CABABF65-F035-44F2-BAA6-EC3CE6A94E35   
          2022-04-26 08:37:59.4870000  8161D78C-752C-4375-A4BE-AA0023AB3ECB   
          2022-04-26 10:12:29.9890000  8BA9C6FD-CAE0-4161-AB14-2F08F979E76A   
-20Neg42  2022-04-26 08:26:01.2790000  0F98720E-2559-45BC-9978-7F1266CD4DC7   
          2022-04-26 08:37:59.5340000  860F5A70-D7B8-4D3A-98F6-0E12372999B5   
...                                                                     ...   
ZAS-00422 2022-04-26 15:36:10.6210000  0144198D-7F4A-4306-9CA7-BFB25363FD0B   
ZAS-00423 2022-04-26 15:40:32.8270000  46289EC2-7CE2-418B-88EA-2E487BF141A1   
ZAS-00425 2022-04-26 15:40:32.8740000  A10C5416-1877-4EDC-9094-F51740859687   
ZAS-00426 2022-04-26 15:40:32.9180000  F5B3CBA1-7025-47DE-B590-943BE3D7BCBB   
ZAS-00427 2022-04-26 15:40:32.9560000  FFD9F9E5-CF24-430F-87B1-AD6D78E8AD25   

                                                                  Test Guid  \
Sample ID Start Date/Time                                                     
-20Neg41  2022-04-26 08:26:01.2280000  3430b4bc-29c5-ec11-85fa-5cf370923501   
          2022-04-26 08:37:59.4870000  e3c17269-2bc5-ec11-85fa-5cf370923501   
          2022-04-26 10:12:29.9890000  8fd6549d-38c5-ec11-85fa-5cf370923501   
-20Neg42  2022-04-26 08:26:01.2790000  3d30b4bc-29c5-ec11-85fa-5cf370923501   
          2022-04-26 08:37:59.5340000  ecc17269-2bc5-ec11-85fa-5cf370923501   
...                                                                     ...   
ZAS-00422 2022-04-26 15:36:10.6210000  ef4ddbd4-65c5-ec11-85fa-5cf370923501   
ZAS-00423 2022-04-26 15:40:32.8270000  814c3871-66c5-ec11-85fa-5cf370923501   
ZAS-00425 2022-04-26 15:40:32.8740000  8a4c3871-66c5-ec11-85fa-5cf370923501   
ZAS-00426 2022-04-26 15:40:32.9180000  934c3871-66c5-ec11-85fa-5cf370923501   
ZAS-00427 2022-04-26 15:40:32.9560000  9c4c3871-66c5-ec11-85fa-5cf370923501   

                                      Software Version Assay Version  \
Sample ID Start Date/Time                                              
-20Neg41  2022-04-26 08:26:01.2280000          1.9.2.6         4.2.1   
          2022-04-26 08:37:59.4870000          1.9.2.6         4.2.1   
          2022-04-26 10:12:29.9890000          1.9.2.6         4.2.1   
-20Neg42  2022-04-26 08:26:01.2790000          1.9.2.6         4.2.1   
          2022-04-26 08:37:59.5340000          1.9.2.6         4.2.1   
...                                                ...           ...   
ZAS-00422 2022-04-26 15:36:10.6210000          1.9.2.6         4.2.1   
ZAS-00423 2022-04-26 15:40:32.8270000          1.9.2.6         4.2.1   
ZAS-00425 2022-04-26 15:40:32.8740000          1.9.2.6         4.2.1   
ZAS-00426 2022-04-26 15:40:32.9180000          1.9.2.6         4.2.1   
ZAS-00427 2022-04-26 15:40:32.9560000          1.9.2.6         4.2.1   

                                      N500 Serial Number PCR Module Serial  \
Sample ID Start Date/Time                                                    
-20Neg41  2022-04-26 08:26:01.2280000           96000007              V104   
          2022-04-26 08:37:59.4870000           96000007              V153   
          2022-04-26 10:12:29.9890000           96000007              V104   
-20Neg42  2022-04-26 08:26:01.2790000           96000007              V104   
          2022-04-26 08:37:59.5340000           96000007              V153   
...                                                  ...               ...   
ZAS-00422 2022-04-26 15:36:10.6210000           96000007              V153   
ZAS-00423 2022-04-26 15:40:32.8270000           96000007              V153   
ZAS-00425 2022-04-26 15:40:32.8740000           96000007              V153   
ZAS-00426 2022-04-26 15:40:32.9180000           96000007              V153   
ZAS-00427 2022-04-26 15:40:32.9560000           96000007              V153   

                                       Pcr Cartridge Lane 

In [205]:
##Step 7: Get Channel Data for Original Data

config = dict(server='ABR-RIPLEYA-D1',
    port=      1433,
    database=  'neumodxRawDataDB',
    )

conn_str = ('SERVER={server};' +
            'Database={database};' +
            'TRUSTED_CONNECTION=yes')

conn = pyodbc.connect(
    r'DRIVER={SQL Server};' +
    conn_str.format(**config)
    )

def getSQLData_channel_summary(environment, testGuids):
    query = ("Select * from [dbo].[channel_summary]"+
             "where  [Environment] like '%"+environment+"%'")
    print("Query:", query)
    df = pd.read_sql(query, conn)
    print("Length of Raw Data Frame: "+ str(len(df)))
    
    df.set_index('Test Guid', inplace=True)
    df = df.loc[testGuids,:]
    df.reset_index(inplace=True)
    df.drop_duplicates(['Test Guid', 'Channel'], inplace=True)
    print("Length of Filtered Data Frame: "+ str(len(df)))
    df.set_index(['Test Guid', 'Channel'],inplace=True)
    return df

ChannelDataOriginal = getSQLData_channel_summary(environment, [x for x in ChainOfCustodyInfo['Test Guid'].unique().tolist() if pd.isnull(x)==False])
ChannelDataOriginal['Localized Result'] = np.where(ChannelDataOriginal['Localized Result'].isnull(), ChannelDataOriginal['Target Result'], ChannelDataOriginal['Localized Result'])

##Step 8: Subset ChannelDataOriginal to include only fields that are relavant to describing the amplification state.
ChannelDataOriginal = ChannelDataOriginal[['Localized Result', 'Ct', 'End Point Fluorescence', 'Max Peak Height', 'EPR', 'Baseline First Cycle', 'Baseline Last Cycle', 'Baseline Slope']]


Query: Select * from [dbo].[channel_summary]where  [Environment] like '%HPV_RPT-8430_Original_Data_3%'
Length of Raw Data Frame: 76680
Length of Filtered Data Frame: 21536


In [206]:
##Step 9: Merge together ChainOfCustodyInfo with ChannelDataOriginal for one common dataframe to combine with the Line Data.
ChainOfCustodyInfo.dropna(subset=['Test Guid'],inplace=True)
ChainOfCustodyInfo.drop_duplicates('Test Guid',inplace=True)
OriginalDataAll = ChannelDataOriginal.join(ChainOfCustodyInfo.set_index('Test Guid'))

In [207]:
##Step 10: Rename Channels in OriginalDataAll for merge with Line Data, annotate other columns to distinguish them as "Original" Columns
OriginalDataAll = OriginalDataAll.reset_index().replace({'Green':'Green_470_510',
                                                         'Yellow':'Yellow_530_555',
                                                         'Red':'Red_625_660',
                                                         'Orange':'Orange_585_610'}).set_index(['SampleUId','Channel'])
for col in OriginalDataAll.columns:
    OriginalDataAll.rename({col:'Original '+col}, axis=1, inplace=True)

In [208]:
##Step 11: Read LineData from CSV, Merge with OriginalDataAll, and Export to CSV.
lineData = pd.read_csv('P:/Users/Aaron/HPV Data Analysis/RPT-8430_CompiledData_Final.csv')#.set_index(["Sample ID", 'StartDateTime'],inplace=True)
lineData.rename({'SampleID':'Sample ID', 'StartDateTime':'Start Date/Time'}, axis=1, inplace=True)
lineData.reset_index(inplace=True)
lineData['SampleUId'] = lineData['SampleUId'].str.upper()
lineData.set_index(['SampleUId','Channel'],inplace=True)

FinalData = lineData.join(OriginalDataAll, rsuffix='_Original')
FinalData[~FinalData['Original N500 Serial Number'].isnull()].to_csv('RPT_8430_FinalData_with_OriginalData.csv')

In [209]:
FinalData.reset_index(inplace=True)


In [210]:
FinalData.set_index(['Channel', 'ProcessingStep'],inplace=True)

In [211]:
def checkCartridgeNeighbors(data, cartridgeField='CartridgeId', cartridgeLaneField='PcrCartridgeLane', resultField='LocalizedResult'):
    """
    Finds and returns the specified field result associated from the neighboring cartridge lanes (left & right) of each sample included in the DataFrame.
    Parameters
    ----------
    data: pandas.DataFrame
        Pandas Dataframe used as input.
    cartridgeField: str
        Field used to identify the NeuMoDx Cartridge used for processing in data.
    cartridgeLaneField: str
        Field used to identify the Cartridge Lane used for sample processing in data.
    resultField: str
        Field value to return from Neighboring Lanes.
    """

    ##Determine Original Index
    original_index = data.index.names
    

    ##Reset Index (to ensure index is always common no matter what input).
    data.reset_index(inplace=True)

    ##Add "LeftNeighbor" & "RightNeighbor" Result Columns
    data.loc[:, 'LeftNeighbor'+resultField] = np.nan
    data.loc[:, 'RightNeighbor'+resultField] = np.nan

    for idx in data.index:

        ##Get Cartridge Barcode & Lane:
        cartridgeid = data.loc[idx, cartridgeField]
        lane = data.loc[idx, cartridgeLaneField]
        channel = data.loc[idx, 'Channel']


        ##Find Left Cartridge Neighbor & Get Result:
        leftNeighbor = data.loc[((data[cartridgeField]==cartridgeid)&
                                           (data[cartridgeLaneField]==lane-1)&
                                           (data['Channel']==channel)),:]
        
        if len(leftNeighbor)>0:
            leftNeighborResult = leftNeighbor[resultField].values[0]
        else:
            leftNeighborResult = np.nan

        ##Find Right Cartridge Neighbor & Get Result:

        rightNeighbor = data.loc[((data[cartridgeField]==cartridgeid)&
                                           (data[cartridgeLaneField]==lane+1)&
                                           (data['Channel']==channel)),:]
        
        if len(rightNeighbor)>0:
            rightNeighborResult = rightNeighbor[resultField].values[0]
        else:
            rightNeighborResult = np.nan
        
        ##Fill in LeftNeighbor and RightNeighbor Results into appropriate DataFrame Columns
        data.loc[idx, 'LeftNeighbor'+resultField] = leftNeighborResult
        data.loc[idx, 'RightNeighbor'+resultField] = rightNeighborResult

    data.set_index(original_index, inplace=True)
    return data




        
        
    
    


In [212]:
YellowRawData = checkCartridgeNeighbors(FinalData.loc[('Yellow_530_555','Raw'),:].sort_values(['Original Pcr Cartridge Barcode', 'Original Pcr Cartridge Lane']), cartridgeField='Original Pcr Cartridge Barcode', cartridgeLaneField='Original Pcr Cartridge Lane')


In [213]:
YellowRawData = checkCartridgeNeighbors(YellowRawData.sort_values(['Original Pcr Cartridge Barcode', 'Original Pcr Cartridge Lane']), cartridgeField='Original Pcr Cartridge Barcode', cartridgeLaneField='Original Pcr Cartridge Lane')


In [215]:
YellowRawData = YellowRawData[~YellowRawData['Original N500 Serial Number'].isnull()]

In [221]:
YellowRawData[['Original Software Version', 'Original Assay Version', 'Sample ID']].groupby(['Original Software Version', 'Original Assay Version']).count().sort_index()

Sample ID
Original Software Version Original Assay Version           
1.7.3.10                  9.0.17                         52
1.7.3.8                   9.0.17                       1504
                          9.0.27                        116
1.7.3.9                   9.0.17                       2862
1.8.3.5                   4.1.0                         387
1.9.2.6                   4.2.1                         454

In [232]:
YellowRawData[['AssayVersion', 'ReplicateNumber']].groupby(['AssayVersion']).count().sort_index()

,ReplicateNumber
AssayVersion,
4.2.1,5384


In [231]:
YellowRawData[['NeuMoDxSoftwareVersion', 'AssayVersion']]

NeuMoDxSoftwareVersion AssayVersion
Channel        ProcessingStep                                    
Yellow_530_555 Raw                               NaN        4.2.1
               Raw                               NaN        4.2.1
               Raw                               NaN        4.2.1
               Raw                               NaN        4.2.1
               Raw                               NaN        4.2.1
...                                              ...          ...
               Raw                               NaN        4.2.1
               Raw                               NaN        4.2.1
               Raw                               NaN        4.2.1
               Raw                               NaN        4.2.1
               Raw                               NaN        4.2.1

[5384 rows x 2 columns]